# Final project Georg Hausbichler

Before we can start, we need to load the SQL-extension and create a database for the project, in which the information that will be loaded, altered and created is stored.

In [1]:
%reload_ext sql

In [2]:
%sql sqlite:///project.db

### Importing CSV data

In [35]:
# deleting the table we are about to create, to ensure that the project can be run as a whole multiple times 
%sql drop table if exists players;

 * sqlite:///project.db
Done.


[]

The dataset is available on Kaggle as "NBA Players" in the form of a CSV-file. It lists the players of the NBA (National Basketball Association) for every season from 1996 to 2022. 

It features an entry for every player for every season including a unique index, their name and 20 additional columns with personal and playing data.

In [36]:
# importing pandas in order to import csv data
import pandas as pd
# importing sqlite3 to create connection object to data base
import sqlite3 

conn = sqlite3.connect("project.db") 

# loading the players.csv file (includes player data for all NBA-players from 1996 - 2022)
# https://www.kaggle.com/datasets/justinas/nba-players-data/data
pd.read_csv("./players.csv").to_sql("players", conn, index=False)

12844

In [37]:
%sql select * from players limit 10;

 * sqlite:///project.db
Done.


Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,2,42,64,3.9,1.5,2.4,0.3,0.042,0.071,0.1689999999999999,0.487,0.248,1996-97
1,Gaylon Nickerson,WAS,28.0,190.5,86.18248,Northwestern Oklahoma,USA,1994,2,34,4,3.8,1.3,0.3,8.9,0.03,0.111,0.174,0.497,0.043,1996-97
2,George Lynch,VAN,26.0,203.2,103.418976,North Carolina,USA,1993,1,12,41,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996-97
3,George McCloud,LAL,30.0,203.2,102.0582,Florida State,USA,1989,1,7,64,10.2,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996-97
4,George Zidek,DEN,23.0,213.36,119.748288,UCLA,USA,1995,1,22,52,2.8,1.7,0.3,-14.1,0.102,0.1689999999999999,0.195,0.5,0.064,1996-97
5,Gerald Wilkins,ORL,33.0,198.12,102.0582,Tennessee-Chattanooga,USA,1985,2,47,80,10.6,2.2,2.2,-5.8,0.031,0.064,0.203,0.503,0.143,1996-97
6,Gheorghe Muresan,WAS,26.0,231.14,137.438376,None,USA,1993,2,30,73,10.6,6.6,0.4,6.9,0.098,0.217,0.185,0.618,0.024,1996-97
7,Glen Rice,CHH,30.0,203.2,99.79024,Michigan,USA,1989,1,4,79,26.8,4.0,2.0,3.2,0.025,0.087,0.272,0.605,0.088,1996-97
8,Glenn Robinson,MIL,24.0,200.66,106.59412,Purdue,USA,1994,1,1,80,21.1,6.3,3.1,-2.9,0.051,0.144,0.278,0.528,0.146,1996-97
9,Grant Hill,DET,24.0,203.2,102.0582,Duke,USA,1994,1,3,80,21.4,9.0,7.3,6.9,0.049,0.2319999999999999,0.283,0.556,0.356,1996-97


Because the first column containing the index for every player for every season is unnamed, we will rename it to "index". 

In [6]:
%%sql

alter table players
rename column "Unnamed: 0" to
"index";

 * sqlite:///project.db
Done.


[]

### Creating the "careers"-table

The information on player's career lengths is not included because the data is per season.

However, player's career lengths can be calculated by counting the number of entries each player has in the table, which we will do and save this information in a new table called "careers". Through the additional ordering-statement the table shows the players who had the longest careers in this timeframe first.

In [7]:
%%sql

drop table if exists careers;
create table careers as select 
player_name, count(*) as total_seasons from players group by player_name
order by total_seasons desc;


 * sqlite:///project.db
Done.
Done.


[]

In [8]:
%sql select * from careers limit 10;

 * sqlite:///project.db
Done.


player_name,total_seasons
Vince Carter,22
Dirk Nowitzki,21
Udonis Haslem,20
LeBron James,20
Kobe Bryant,20
Kevin Garnett,20
Jamal Crawford,20
Tyson Chandler,19
Tim Duncan,19
Paul Pierce,19


We can expand this table to also include the total number of teams every player played for as well as the career averages for major statistics (points, assists and rebounds). 

This can be achieved by counting the number of distinct team abbreviations for every player and then firstly calculating the sum of the statistics, dividing it by every row this player appears in and finally rounding the result to improve its appearance. 

Through ordering primarily by total seasons and secondarily by total points we can find the players who had the longest careers and averaged the most points between 1996 and 2022.

In [9]:
%%sql

drop table if exists careers;

create table careers as select 
player_name, 
count(*) as total_seasons,
count(distinct(team_abbreviation)) as total_teams,
round(sum(pts)/count(*), 2) as career_ppg,
round(sum(reb)/count(*), 2) as career_rpg,
round(sum(ast)/count(*), 2) as career_apg
from players group by player_name
order by total_seasons desc, career_ppg desc;

 * sqlite:///project.db
Done.
Done.


[]

In [10]:
%sql select * from careers limit 10;

 * sqlite:///project.db
Done.


player_name,total_seasons,total_teams,career_ppg,career_rpg,career_apg
Vince Carter,22,8,16.35,4.23,2.97
Dirk Nowitzki,21,1,20.11,7.35,2.34
LeBron James,20,3,27.2,7.54,7.34
Kobe Bryant,20,1,24.2,5.2,4.76
Kevin Garnett,20,3,17.35,9.83,3.63
Jamal Crawford,20,9,13.93,2.08,3.36
Udonis Haslem,20,1,5.83,5.09,0.62
Carmelo Anthony,19,6,22.02,6.16,2.64
Paul Pierce,19,4,18.93,5.44,3.36
Tim Duncan,19,1,18.89,10.77,3.01


In [11]:
%sql select count(*) from careers;

 * sqlite:///project.db
Done.


count(*)
2551


The table was successfully overwritten to now include multiple new interesting statistics about the players. 

It contains information about the careers of 2551 NBA-players.

### Importing RDF sources

In [12]:
# installing the necessary package
#pip install SPARQLWrapper 

In [13]:
# importing the packages needed and creating a connection to dbpedia
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

The following Sparql-query selects the source, the label (in this case the player name) and the birthplace of all objects on DBpedia which are of the type "BasketballPlayer". By filtering only for entries of which the language of the player name and birthplace are in english, we eliminate the redundancy of getting every result in multiple languages.

I chose to write the query in this form because there is no specific type "NBAPlayer". However, this leads to the result including many people that never played in the NBA (for example female players, international players, college players, ...).

In [14]:
sparql.setQuery("""
SELECT ?s ?l ?b WHERE {
?s a dbo:BasketballPlayer.
?s rdfs:label ?l.
?s dbo:birthPlace ?p.
?p rdfs:label ?b.
FILTER (lang(?l)='en' && lang(?b)='en').
} 
""")

We now set the format of the result from the query to JSON, execute the query and store its results in a variable in JSON-format. 

In [15]:
sparql.setReturnFormat(JSON)
players_birthplace = sparql.query().convert()["results"]["bindings"]
for res in players_birthplace[:10]:
    print(res)

{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Bill_Hougland'}, 'l': {'type': 'literal', 'xml:lang': 'en', 'value': 'Bill Hougland'}, 'b': {'type': 'literal', 'xml:lang': 'en', 'value': 'Caldwell, Kansas'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Carlisle_Towery'}, 'l': {'type': 'literal', 'xml:lang': 'en', 'value': 'Carlisle Towery'}, 'b': {'type': 'literal', 'xml:lang': 'en', 'value': 'Caldwell County, Kentucky'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Dejan_Vasiljevic'}, 'l': {'type': 'literal', 'xml:lang': 'en', 'value': 'Dejan Vasiljevic'}, 'b': {'type': 'literal', 'xml:lang': 'en', 'value': 'Calgary'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Quincy_Davis_(basketball)'}, 'l': {'type': 'literal', 'xml:lang': 'en', 'value': 'Quincy Davis (basketball)'}, 'b': {'type': 'literal', 'xml:lang': 'en', 'value': 'California'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Bassel_Bawji'}, 'l': {'type': 'l

### Getting the RDF data into the database

We now need to store the result of the Sparql-query as a table in the SQL database in order to be able to combine it with the previously created tables. For this purpose we will utilize pandas capabilities of normalizing JSON-data and then transforming it into an SQL-table.

In [16]:
import pandas as pd
pd.json_normalize(players_birthplace).to_sql('players_birthplace', 
                                 conn, 
                                 if_exists='replace',
                                 index=False)

10000

In [17]:
%%sql

select * from players_birthplace limit 10;

 * sqlite:///project.db
Done.


s.type,s.value,l.type,l.xml:lang,l.value,b.type,b.xml:lang,b.value
uri,http://dbpedia.org/resource/Bill_Hougland,literal,en,Bill Hougland,literal,en,"Caldwell, Kansas"
uri,http://dbpedia.org/resource/Carlisle_Towery,literal,en,Carlisle Towery,literal,en,"Caldwell County, Kentucky"
uri,http://dbpedia.org/resource/Dejan_Vasiljevic,literal,en,Dejan Vasiljevic,literal,en,Calgary
uri,http://dbpedia.org/resource/Quincy_Davis_(basketball),literal,en,Quincy Davis (basketball),literal,en,California
uri,http://dbpedia.org/resource/Bassel_Bawji,literal,en,Bassel Bawji,literal,en,California
uri,http://dbpedia.org/resource/Bill_Sweek,literal,en,Bill Sweek,literal,en,California
uri,http://dbpedia.org/resource/Benjie_Paras,literal,en,Benjie Paras,literal,en,Caloocan
uri,http://dbpedia.org/resource/Bill_Hewitt_(basketball),literal,en,Bill Hewitt (basketball),literal,en,"Cambridge, Massachusetts"
uri,"http://dbpedia.org/resource/Bob_Armstrong_(basketball,_born_1920)",literal,en,"Bob Armstrong (basketball, born 1920)",literal,en,"Cambridge, Ohio"
uri,http://dbpedia.org/resource/Billy_Thompson_(basketball),literal,en,Billy Thompson (basketball),literal,en,"Camden, New Jersey"


Because this table includes a lot of unnecessary columns, we can reduce it by creating a new one just including the essential ones and renaming them.

In [18]:
%%sql

drop table if exists players_birthplace2;

create table players_birthplace2 as select "l.value" as player_name, 
"b.value" as birthplace from players_birthplace;

 * sqlite:///project.db
Done.
Done.


[]

In [19]:
%sql select * from players_birthplace2 limit 10;

 * sqlite:///project.db
Done.


player_name,birthplace
Bill Hougland,"Caldwell, Kansas"
Carlisle Towery,"Caldwell County, Kentucky"
Dejan Vasiljevic,Calgary
Quincy Davis (basketball),California
Bassel Bawji,California
Bill Sweek,California
Benjie Paras,Caloocan
Bill Hewitt (basketball),"Cambridge, Massachusetts"
"Bob Armstrong (basketball, born 1920)","Cambridge, Ohio"
Billy Thompson (basketball),"Camden, New Jersey"


In [20]:
%sql select count(*) from players_birthplace2;

 * sqlite:///project.db
Done.


count(*)
10000


The resulting table stores information of 10000 basketball players' names and birthplaces.

### Combining the data

We will now combine these two tables into one via a left join on the player's name. 
This left join is used here to enhance the careers-table with a new column containing the birthplace of the player.

In [21]:
%%sql

drop table if exists careers_birthplace;

create table careers_birthplace as 
select * from careers
left join players_birthplace2
on careers.player_name = players_birthplace2.player_name;

 * sqlite:///project.db
Done.
Done.


[]

In [29]:
%%sql

select * from careers_birthplace limit 20;

 * sqlite:///project.db
Done.


player_name,total_seasons,total_teams,career_ppg,career_rpg,career_apg,player_name:1,birthplace
Vince Carter,22,8,16.35,4.23,2.97,Vince Carter,"Daytona Beach, Florida"
Dirk Nowitzki,21,1,20.11,7.35,2.34,None,None
LeBron James,20,3,27.2,7.54,7.34,None,None
Kobe Bryant,20,1,24.2,5.2,4.76,None,None
Kevin Garnett,20,3,17.35,9.83,3.63,None,None
Jamal Crawford,20,9,13.93,2.08,3.36,None,None
Udonis Haslem,20,1,5.83,5.09,0.62,Udonis Haslem,"Miami, Florida"
Carmelo Anthony,19,6,22.02,6.16,2.64,Carmelo Anthony,2010–11 New York Knicks season
Carmelo Anthony,19,6,22.02,6.16,2.64,Carmelo Anthony,New York City
Paul Pierce,19,4,18.93,5.44,3.36,Paul Pierce,"Oakland, California"


Joining the two tables worked, however there are some issues with the result. 

As you can see from the first 20 rows above, the players Carmelo Anthony, Tim Duncan and Pau Gasol now appear in two rows with different values in the "birthplace"-column. This happened because for some players the birthplace-value is stored incorrectly on DBpedia and actually includes multiple values which sometimes do not represent the correct birthplace of the player. 

For example, for the player Carmelo Anthony there are two values stored on DBpedia as his birthplace: "New York City" and "2010-11 New York Knicks season".

In [23]:
%%sql

select count(*) from careers_birthplace limit 10;

 * sqlite:///project.db
Done.


count(*)
2651


This mistake occured 103 times while joining the tables, as the careers_birthplace-table has 103 rows more than the original careers-table. This means there are 103 duplicated careers, each having a different entry in the birthplace-column.

We can simplify the table by recreating it and leaving out the unwanted columns.

In [24]:
%%sql 

drop table if exists careers_birthplace2;
create table careers_birthplace2 as
select player_name, total_seasons, total_teams, career_ppg, career_rpg, career_apg, birthplace 
from careers_birthplace;

select * from careers_birthplace2 limit 20;

 * sqlite:///project.db
Done.
Done.
Done.


player_name,total_seasons,total_teams,career_ppg,career_rpg,career_apg,birthplace
Vince Carter,22,8,16.35,4.23,2.97,"Daytona Beach, Florida"
Dirk Nowitzki,21,1,20.11,7.35,2.34,None
LeBron James,20,3,27.2,7.54,7.34,None
Kobe Bryant,20,1,24.2,5.2,4.76,None
Kevin Garnett,20,3,17.35,9.83,3.63,None
Jamal Crawford,20,9,13.93,2.08,3.36,None
Udonis Haslem,20,1,5.83,5.09,0.62,"Miami, Florida"
Carmelo Anthony,19,6,22.02,6.16,2.64,2010–11 New York Knicks season
Carmelo Anthony,19,6,22.02,6.16,2.64,New York City
Paul Pierce,19,4,18.93,5.44,3.36,"Oakland, California"


### Saving the result as a CSV

As a final step, we will store our result as a CSV file. We will utilize the capabilities of pandas once again.

For this purpose, we have to create a SQL-Alchemy connection to the database in order to transform the SQL-table to a Pandas dataframe. After that, we can turn that dataframe into a CSV file, which is automatically saved in the directory. 

In [25]:
# importing the module necessary
from sqlalchemy import create_engine
 
# SQLAlchemy connection to our database
cn = create_engine('sqlite:///project.db').connect()

In [26]:
df = pd.read_sql_table('careers_birthplace2', cn)
df.head()

,player_name,total_seasons,total_teams,career_ppg,career_rpg,career_apg,birthplace
0,Vince Carter,22,8,16.35,4.23,2.97,"Daytona Beach, Florida"
1,Dirk Nowitzki,21,1,20.11,7.35,2.34,None
2,LeBron James,20,3,27.20,7.54,7.34,None
3,Kobe Bryant,20,1,24.20,5.20,4.76,None
4,Kevin Garnett,20,3,17.35,9.83,3.63,None


In [27]:
df.to_csv("nba_careers_final.csv")